In [1]:
!pip install python-dotenv
!pip install openai

# Getting data from Mergeflow's API

In [2]:
import requests
import json
import os
from dotenv import load_dotenv

load_dotenv()  # Load variables from .env file
mergeflow_api_key = os.getenv('MERGEFLOW_API_KEY')

# the API call
query = 'factory automation' # topic is "factory automation"
dataset = '&sp=3814' # dataset: industry news
rows = str(3) # first 3 docs only

# in the url below, replace "fw" by your user name. You can see your user name at mergeflow.net
# -> Account -> the string in angle brackets
url = 'https://mergeflow.net/api/v1/leonard/search/getContent?q=' + query + dataset + '&rows=' + rows

# Define the headers to be sent with the request
headers = {
    'MergeflowNet-Api-Auth-Key': mergeflow_api_key
}

# Get and print the response
response = requests.post(url, headers=headers)

mergeflow_docs = []

# Check the response status code and content
if response.status_code == 200:
    mergeflow_response_json = response.json()
    
    #pretty-print the JSON response -- useful to understand what the JSON looks like
    #pretty_json = json.dumps(mergeflow_response_json, indent=4)
    #print(pretty_json)
    
    # get title, URL, date, and content
    for document in mergeflow_response_json['Documents']:
        current_doc = {
            'title': document['Title'],
            'date': document['Date'],
            'url': document['Url'],
            'content': document['Content'][:5000]
        }
        
        mergeflow_docs.append(current_doc)
        #print(content)        

    print('Found ' + str(len(mergeflow_docs)) + ' documents.')

else:
    print('Request failed with status code:', response.status_code)
    print(response.text)  # Print the response text


Found 3 documents.


# Answering questions on the results with OpenAI GPT

In [39]:
import openai

openai.api_key = os.getenv('OPENAI_API_KEY')

system_prompt = f"""

You are Senior Vice President Technology and Innovation & CTO of Factory Automation. When you evaluate a new business idea, you strictly adhere to the following guiding principles:
        - Focus on Sense&Act.
        - Prioritize quick, low-investment idea testing.
        - Aim for large profits with smaller budgets, avoiding investments over 10-15 million per portfolio element.
        - Target a minimum 10 million EUR annual revenue.
        - Specialize in LLMs in Engineering and Operations.
        - Avoid new ecosystem creation.
        - Core business: Sense&Act, PMA, Scada, PLCs.
        - Prioritize product over solution business for scalability.
        - Consider AI as a potential product.
        - Ensure all initiatives align with business profitability.
        - Make investment decisions as if using personal funds.
        - Aim for non-cannibalizing, fringe markets.
        - PLCs are scalable; seek similar scalability in AI applications.
        - Validate ideas with business units and customer feedback.
        - Avoid focusing solely on infrastructure to ensure profitable outcomes.
        
        """
# Initialize an empty list to store problem messages
problems_list = []

for doc in mergeflow_docs:

    user_prompt1 = f"""
    Consider the text below, delimited by ```, in the area of {query}:
    ```
    {doc['content']}
    ```
        I'm interested in exploring problems that will occur in the area of {query} using the concept of the adjacent possible, as outlined by Stuart Kauffman and popularized by Steven Johnson. This approach emphasizes incremental innovation by considering what's just beyond the current technological, cultural, or conceptual boundaries, yet still achievable with existing resources or slight advancements. 
        
        I'm looking for ideas that are not far-fetched but are innovative problems that could occur as a result of what currently exists, tapping into the untapped potential lying just outside the present scope of {query}.

        Given this framework, please analyze the current trends, technologies, consumer behaviors, and market needs in {query}. Identify where there might be gaps or unmet needs that align with the concept of the adjacent possible. Consider the following:

        Technological Advancements: What recent technological developments have occurred, and how might they open new opportunities for innovation or improvement in factory automation?
        Consumer Behavior Shifts: How have consumer preferences and behaviors evolved recently, and what does this imply about potential future needs or desires?
        Regulatory Changes: Are there any recent or upcoming changes in regulations that might create new opportunities or necessitate innovations in factory automation?
        Cross-industry Inspiration: Look at how other industries are innovating. Are there any ideas or technologies that could be adapted or extended to fit the context of factory automation?

        For the problem, please output: 
        - The problem (max 50 words)
        - Which user group is impacted by this problem the most (max 25 words)
        - What is the specific pain point for this user group resulting from the problem (max 50 words)
    
    """
    
    completion = openai.chat.completions.create(
        model="gpt-3.5-turbo-0125",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt1}
        ],
        temperature=0.7,
        max_tokens=2000,
        top_p=0.95,
        frequency_penalty=0,
        presence_penalty=0,
        stop=None
    )
    
    # Append the completion message content to the problems_list
    problems_list.append(completion.choices[0].message.content)
    
    # This prints the content of the output in a more readable way
    print(completion.choices[0].message.content)
    print("\n---\n")  # This adds a separator between the content blocks for readability
    
    
    
    
# second agent:

# Now, iterate over problems_list to rate each problem
rated_problems_list = []
for problem in problems_list:
    user_prompt2 = f"""
    Before you give me any output, please rate the problems according to the following criteria. Please be very critical in your ratings and only give limited number of 3:
        - Size of user group (how many of the mentioned customers exist) (1 = small target group, 2 = medium target group, 3 = large target group)
        - Relevancy of pain point to the specific industry of {query} and the respective user group (e.g., patty not relevant for factory automation) (1 = not relevant to users, 2 = medium relevant to users, 3 = very relevant to users)
        - How specific is the problem? (1 = very generic problem, 2 = medium generic, 3 = specific and not generic problem)

    Given the problem:
    {problem}

    Please repeat the name of the problem and then output the ratings and a one-sentence justification (max 20 words).
    Add up the three ratings you gave to an overall score out of 9.
    
    """
    
    completion = openai.chat.completions.create(
        model="gpt-3.5-turbo-0125",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt2}
        ],
        temperature=0.7,
        max_tokens=2000,
        top_p=0.95,
        frequency_penalty=0,
        presence_penalty=0,
        stop=None
    )

    # Assume we store the rated problem details for later use
    rated_problems_list.append(completion.choices[0].message.content)

    # Print the rated problem for immediate feedback
    print(completion.choices[0].message.content)
    print("\n---\n")
    

# third agent 

user_prompt3 = f"""

    Consider the following problems: {problems_list}
    Consider the corresponding ratings: {rated_problems_list}
    
    Please output the problem with the highest overall rating in the {rated_problems_list}, including: 
        - The problem (max 50 words)
        - Which user group is impacted by this problem the most (max 25 words)
        - What is the specific pain point for this user group resulting from the problem (max 50 words)
        - One sentence that summarises the three selection criteria (Size of user group / Relevancy of pain point to the specific industry of {query} and the respective user group / How specific is the problem)


    """
    
completion = openai.chat.completions.create(
        model="gpt-3.5-turbo-0125",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt3}
        ],
        temperature=0.7,
        max_tokens=2000,
        top_p=0.95,
        frequency_penalty=0,
        presence_penalty=0,
        stop=None
    )


# Print the rated problem for immediate feedback
print(completion.choices[0].message.content)
print("\n---\n")




- **Problem:** Lack of personalized and adaptive automation systems in factory automation, limiting efficiency and flexibility in production processes.
  
- **User Group:** Manufacturing workers and supervisors in factories.

- **Pain Point:** Workers and supervisors face challenges in optimizing workflows and adapting to changing production needs due to rigid and non-adaptive automation systems, leading to inefficiencies, downtime, and potential errors in operations.

---

- **Problem:** Integrating private 5G networks and AI technology into factory automation systems to enhance connectivity, data processing, and operational efficiency.

- **User Group:** Factory managers and engineers in charge of optimizing production processes and equipment performance.

- **Pain Point:** Difficulty in seamlessly incorporating complex private 5G and AI solutions into existing automation systems, leading to integration challenges, potential downtime, and the need for specialized training to maximize